In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install sentence-transformers chromadb torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s 

In [ ]:
import hashlib

def hash_file(filepath):
    md5 = hashlib.md5()
    with open(filepath, 'rb') as f:
        while chunk := f.read(8192):
            md5.update(chunk)
    return md5.hexdigest()

In [ ]:
import requests

# URL của API
url = "https://hybrid-analysis.com/api/v2/search/hash"

headers = {
    "accept": "application/json",
    "api-key": "xxx", 
    "Content-Type": "application/x-www-form-urlencoded",
}



def query_by_hash(hash):
    data = {
        "hash": hash, 
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print(f"Request failed with status code {response.status_code}")
        return None


def search_hash(hash, apt_group):
    # URL của API
    url = "https://hybrid-analysis.com/api/v2/search/hash"

    headers = {
        "accept": "application/json",
        "api-key": "xxx", 
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {
        "hash": hash,  # Thay bằng hash muốn query
    }
    filtered_result = {}
    filtered_result[hash] = {}
    filtered_result[hash]['apt_group'] = apt_group
    filtered_result[hash]['ttp'] = {}
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        result = response.json()

        for i in range(len(result)):
          if result[i]['signatures'] != []:
            for j in range(len(result[i]['signatures'])):
                if result[i]['signatures'][j]['attck_id'] != None:
                    if result[i]['signatures'][j]['attck_id'] not in list(filtered_result[hash]['ttp'].keys()):
                      filtered_result[hash]['ttp'][result[i]['signatures'][j]['attck_id']] = []
                    for k in result[i]['signatures'][j]['description'].split("\n"):
                      filtered_result[hash]['ttp'][result[i]['signatures'][j]['attck_id']].append(k)
        return filtered_result
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

In [ ]:

import time
import os
from pprint import pprint
path1 = "/content/drive/MyDrive/DA_NT522/malware_exe"
filepath = []
with open("/content/drive/MyDrive/Tài liệu đồ án chuyên ngành/processed_filepath.json", 'r', encoding='utf-8') as f:
    processed_files = json.load(f)
count = 0
for file_name in os.listdir(path1):
    full_path = os.path.join(path1, file_name)
    if full_path not in processed_files and full_path != "/content/drive/MyDrive/DA_NT522/malware_exe/malware_4951.exe":
      filepath.append(full_path)
behaviors_ttp = []
print(len(filepath))
for filename in filepath:
  hashes = hash_file(filename)
  print("File:", filename)
  print(count)
  count += 1
  result = query_by_hash(hashes)
  print(hashes)
  time.sleep(0.5)
  for i in range(len(result)):
    if len(result[i]['signatures']) != 0:
      for j in range(len(result[i]['signatures'])):
        if result[i]['signatures'][j]['attck_id'] != None:
            for k in result[i]['signatures'][j]['description'].split("\n"):
              sigdict = {}
              sigdict['behavior'] = result[i]['signatures'][j]['name'] + " " + k
              sigdict['ttp'] = result[i]['signatures'][j]['attck_id']
              behaviors_ttp.append(sigdict)
  with open('/content/drive/MyDrive/Tài liệu đồ án chuyên ngành/behaviors_ttp.json', 'w', encoding='utf-8') as f:
    json.dump(behaviors_ttp, f, ensure_ascii=False, indent=2)
  processed_files.append(filename)
  with open("/content/drive/MyDrive/Tài liệu đồ án chuyên ngành/processed_filepath.json", 'w', encoding='utf-8') as f:
      json.dump(processed_files, f, ensure_ascii=False, indent=2)